In [ ]:
import cv2
import numpy as np

#Each marker: name, HSV range, BGR color for drawing, and its list of points
markers = [
    {
        "name": "blue",  # Marker name
        "lower": np.array([100, 127, 10]),  # HSV lower bound for blue
        "upper": np.array([120, 255, 255]),  # HSV upper bound for blue
        "bgr": (255, 0, 0),  # BGR color used to draw blue
        "points": []  # Stores detected points for blue marker
    },
    {
        "name": "yellow",  # Marker name
        "lower": np.array([15, 50, 200]),  # HSV lower bound for yellow
        "upper": np.array([50, 213, 255]),  # HSV upper bound for yellow
        "bgr": (0, 255, 255),  # BGR color used to draw yellow
        "points": []  # Stores detected points for yellow marker
    }
    # Add more markers here in the same format if needed!
]

# Webcam setup
cap = cv2.VideoCapture(0)
cap.set(3, 640)  # Width
cap.set(4, 480)  # Height
cap.set(10, 150) # Brightness

#Function to find the center of a marker cap
def find_marker_center(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # Find all external contours
    if contours: # If at least one contour is found
        largest = max(contours, key=cv2.contourArea) # Take the largest contour
        area = cv2.contourArea(largest)   # Calculate its area
        if area > 500:
            x, y, w, h = cv2.boundingRect(largest)    # Get bounding box of the contour
            cx = x + w // 2        # Calculate center X
            cy = y + h // 2        # Calculate center Y
            return (cx, cy)         # Return center as tuple
    return None  # If nothing valid found, return None 
    
#Main loop
while True:
    success, frame = cap.read()    # Read a frame from the webcam
    if not success:                # If webcam failed, exit loop
        break

    frame = cv2.flip(frame, 1)  # Mirror image for natural interaction
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Process each marker one by one
    for marker in markers:
        mask = cv2.inRange(hsv, marker["lower"], marker["upper"])  # Create a mask for current marker color

        center = find_marker_center(mask)  # Find the center of the marker
        
        if center:  # If a center is found
            marker["points"].append(center)  # Add it to the marker's points list

        # Draw all points found for this marker
        for pt in marker["points"]:
            cv2.circle(frame, pt, 8, marker["bgr"], cv2.FILLED)  # Draw a filled circle at each point
    cv2.imshow("Virtual Painter", frame)
    cv2.imshow("Mask", mask)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
